# Real Estate Content Generator - Demo Interactiva

Este notebook permite probar de forma interactiva la generación de contenido HTML para anuncios inmobiliarios usando OpenAIGenerator, con soporte para varios idiomas y modos de entrada (JSON o formulario).

In [1]:
# Importar librerías necesarias
import json
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from app.llm.openai_generator import OpenAIGenerator
from app.schemas import PropertyInput
from app.generator import generate_content

In [9]:
# Funciones para cargar y procesar el JSON

def cargar_json(archivo):
    try:
        contenido = archivo["content"]
        if isinstance(contenido, memoryview):
            contenido = contenido.tobytes().decode()
        elif isinstance(contenido, bytes):
            contenido = contenido.decode()
        data = json.loads(contenido)
        return data, ""
    except Exception as e:
        return None, f"Error al cargar el JSON: {e}"

def validar_datos(data):
    try:
        property_input = PropertyInput(**data)
        return property_input, ""
    except Exception as e:
        return None, f"Error de validación: {e}"

In [3]:
# Interfaz interactiva: modo JSON o formulario
modo_input = widgets.ToggleButtons(
    options=[('Subir JSON', 'json'), ('Formulario', 'form')],
    value='json',
    description='Modo de entrada:',
    style={'description_width': 'initial'}
)

# Widget para subir JSON
json_uploader = widgets.FileUpload(
    accept='.json',
    multiple=False,
    description='Subir archivo JSON'
)

# Widgets para formulario manual
form_titulo = widgets.Text(description='Título:', style={'description_width': 'initial'})
form_ciudad = widgets.Text(description='Ciudad:')
form_barrio = widgets.Text(description='Barrio:')
form_bedrooms = widgets.BoundedIntText(description='Habitaciones:', min=0, max=20)
form_bathrooms = widgets.BoundedIntText(description='Baños:', min=0, max=20)
form_area = widgets.BoundedFloatText(description='Área (m²):', min=0)
form_balcony = widgets.Checkbox(description='Balcón')
form_parking = widgets.Checkbox(description='Parking')
form_elevator = widgets.Checkbox(description='Ascensor')
form_floor = widgets.BoundedIntText(description='Piso:', min=0, max=100)
form_year = widgets.BoundedIntText(description='Año construcción:', min=1800, max=2100)
form_price = widgets.BoundedFloatText(description='Precio (€):', min=0)
form_listing = widgets.Dropdown(options=[('Venta', 'sale'), ('Alquiler', 'rent')], description='Tipo anuncio:')

In [4]:
# Selector de idioma y modo de generación
selector_idioma = widgets.Dropdown(
    options=[('Español', 'es'), ('Inglés', 'en'), ('Portugués', 'pt')],
    value='es',
    description='Idioma:',
    style={'description_width': 'initial'}
)

selector_modo = widgets.Dropdown(
    options=[('Template', 'template'), ('OpenAI', 'openai'), ('Ollama', 'ollama')],
    value='template',
    description='Modo generación:',
    style={'description_width': 'initial'}
)

In [5]:
# Función para generar el HTML usando el modo y los datos seleccionados
def generar_html(data, modo, idioma):
    import os
    from app.config import settings
    os.environ["GENERATION_MODE"] = modo
    settings.GENERATION_MODE = modo
    data["language"] = idioma
    try:
        property_input = PropertyInput(**data)
        html = generate_content(property_input)
        return html, ""
    except Exception as e:
        return "", f"Error al generar el contenido: {e}"

In [ ]:
# Mostrar la UI interactiva y gestionar eventos
def mostrar_interfaz():
    out = widgets.Output()
    
    def on_generate_clicked(b):
        with out:
            clear_output()
            # Obtener datos según modo
            if modo_input.value == 'json':
                if not json_uploader.value:
                    print('Sube un archivo JSON válido.')
                    return
                archivo = json_uploader.value[0]
                data, err = cargar_json(archivo)
                if err:
                    print(err)
                    return
            else:
                data = {
                    "title": form_titulo.value,
                    "location": {
                        "city": form_ciudad.value,
                        "neighborhood": form_barrio.value
                    },
                    "features": {
                        "bedrooms": form_bedrooms.value,
                        "bathrooms": form_bathrooms.value,
                        "area_sqm": form_area.value,
                        "balcony": form_balcony.value,
                        "parking": form_parking.value,
                        "elevator": form_elevator.value,
                        "floor": form_floor.value,
                        "year_built": form_year.value
                    },
                    "price": form_price.value,
                    "listing_type": form_listing.value,
                    "language": selector_idioma.value
                }
            # Generar HTML
            html, err = generar_html(data, selector_modo.value, selector_idioma.value)
            if err:
                print(err)
                return
            # Mostrar HTML como texto
            print('HTML generado:')
            print('-' * 40)
            print(html)
            print('-' * 40)
            # Mostrar HTML renderizado
            display(HTML('<h4>Vista renderizada:</h4>'))
            display(HTML(html))
    
    # Contenedor para el formulario
    form_box = widgets.VBox([
        form_titulo, form_ciudad, form_barrio, form_bedrooms, form_bathrooms, form_area,
        form_balcony, form_parking, form_elevator, form_floor, form_year, form_price, form_listing
    ])
    
    # Función para mostrar el widget adecuado
    def actualizar_entrada(change=None):
        with out:
            clear_output()
            if modo_input.value == 'json':
                display(json_uploader)
            else:
                display(form_box)
    modo_input.observe(actualizar_entrada, names='value')
    
    boton_generar = widgets.Button(description='Generar contenido', button_style='success')
    boton_generar.on_click(on_generate_clicked)
    
    # Mostrar la UI
    display(widgets.HBox([modo_input, selector_idioma, selector_modo]))
    actualizar_entrada()
    display(boton_generar)
    display(out)

mostrar_interfaz()

Button(button_style='success', description='Generar contenido', style=ButtonStyle())

Output()